In [7815]:
import random

In [7816]:
def generation_poupulation(nombre_individu):
    Population = []
    for _ in range(nombre_individu):
        individu = [random.randint(0,1) for _ in range(8)]
        Population.append(individu)
    return Population

In [7817]:
def evaluate(Population):
    score = []
    for candidate in Population:
        score.append(candidate.count(1))
    return score

# print(evaluate(generation_poupulation()))
# evaluate(Pop)

In [7818]:
def tournament(k, Population):
    """k est la taille du tournoi"""
    tournament_selection = []
    for _ in range(k):
        chosen = random.choice(Population)
        tournament_selection.append(chosen)
    return tournament_selection


aga = tournament(3, generation_poupulation(20))
print(aga)

[[1, 0, 0, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 1, 1]]


In [7819]:
def roulette_wheel(Population):
    wheel = []
    for candidate in Population:
        x = candidate.count(1)
        match x:
            case 8:
                for _ in range(5):
                    wheel.append(candidate)
            case 7:
                for _ in range(4):
                    wheel.append(candidate)
            case 6:
                for _ in range(4):
                    wheel.append(candidate)
            case 5:
                for _ in range(3):
                    wheel.append(candidate)
            case 4:
                for _ in range(3):
                    wheel.append(candidate)
            case 3:
                for _ in range(2):
                    wheel.append(candidate)
            case 2:
                for _ in range(2):
                    wheel.append(candidate)
            case 1:
                for _ in range(1):
                    wheel.append(candidate)
            case 0:
                for _ in range(1):
                    wheel.append(candidate)
            case _:
                print("Error")
    choice = random.choice(wheel)
    return choice

print(roulette_wheel(generation_poupulation(10)))

[0, 1, 1, 0, 1, 1, 0, 0]


In [7820]:
def find_the_best(array):
    best_element = 0
    best_candidate = None
    for index, candidate in enumerate(array):
        if sum(candidate)>best_element:
            best_candidate = index
    return array[best_candidate]

print(find_the_best(aga))

[0, 1, 0, 0, 0, 0, 1, 1]


In [7821]:
def selection_parent(Population,resultat_tournoi):
    best_parent = []
    for _ in range(len(Population)):
        best_parent.append(resultat_tournoi)
    return best_parent

# print(selection_parent(Pop))



In [7822]:
def couple_selection(the_parents, nombre_individu):
    couple_list = []
    for _ in range(int(nombre_individu/2)):
        couple_list.append([random.choice(the_parents),random.choice(the_parents)])
    return couple_list

# print(couple_selection())




In [7823]:
def child_creation(couples):
    #ici les couples vont passé de couple parent à couple enfant
    for couple in couples:
        # effectue le crossover entre les 4 digits centraux de chaque parent
        offsprings = random.randint(1,6)
        temp = couple[1][offsprings:-offsprings]
        couple[1][offsprings:-offsprings] = couple[0][offsprings:-offsprings]
        couple[0][offsprings:-offsprings] = temp
    return couples #retourne les couples enfant

# child_creation(couple_selection())

In [7824]:
def random_mutation(couples):
    muted_children = []
    for duo in couples:
        for child in duo:
            if random.random()>0.995:
                number = random.choice([0,1,2,3,4,5,6,7])
                if child[number] == 0:
                    child[number] = 1
                else:
                    child[number] = 1
            muted_children.append(child)
    return muted_children


# print(random_mutation(child_creation(couple_selection())))
# evaluate(random_mutation(child_creation(couple_selection())))

In [7825]:
def main():
    nombre_individu = 20
    iterations = 50000
    #Generation initiale de la population
    population = generation_poupulation(nombre_individu)
    print(population)
    print("Depart")
    #evaluation des parents
    # evaluation_parent = evaluate(population)
    # print(evaluation_parent)

    #les lignes suivantes : processus de création d'une nouvelle population
    for it in range(iterations):
        # candidate_tourniquet = tournament(3, population)
        # best = find_the_best(candidate_tourniquet)
        best = roulette_wheel(population)
        the_parents = selection_parent(population,best)
        selection_couple =couple_selection(the_parents, nombre_individu)
        the_children = child_creation(selection_couple)
        #mutation_childrens est le groupe d'enfant qui constitue la nouvelle population
        mutation_children = random_mutation(the_children)
        population = mutation_children
        print(population)
        if [1, 1, 1, 1, 1, 1, 1, 1] in mutation_children:
            print(f"MaxOne à l'itération {it}")
            break
        # print(evaluate(population))
        # new_pop = mutation_children
    if [1, 1, 1, 1, 1, 1, 1, 1] not in population:
        print(f"pas de max0ne")

    print(population)
    # evaluation_enfant = evaluate(population)
    # print(evaluation_enfant)

    return None

main()

[[0, 0, 0, 1, 1, 1, 0, 1], [1, 0, 0, 0, 0, 1, 0, 0], [1, 1, 1, 1, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 1, 0], [0, 1, 0, 1, 1, 1, 1, 0], [0, 1, 0, 0, 0, 0, 1, 1], [0, 1, 0, 0, 0, 1, 0, 0], [1, 1, 1, 1, 0, 0, 0, 1], [1, 1, 1, 0, 1, 0, 1, 1], [1, 0, 0, 1, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1], [0, 1, 0, 0, 1, 0, 1, 1], [1, 1, 1, 0, 1, 0, 0, 0], [1, 0, 1, 1, 0, 1, 1, 1], [1, 0, 1, 0, 0, 1, 1, 0], [0, 0, 1, 1, 0, 0, 0, 1], [0, 1, 1, 0, 1, 0, 1, 1], [1, 1, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 0, 1, 1, 0]]
Depart
[[1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1,

Travail sur un Genetic algorithm
